In [1]:
import re
import json
import requests
import pandas as pd
from IPython.display import clear_output


## Get reviews from list of itemids

In [4]:
shop_id = "57629270"
df_items = pd.read_csv("item_ids.csv")
item_ids = df_items['item_id']
# item_ids = ["11355326722"]

# Loop item ids
for idx, item_id in enumerate(item_ids):
    item_id = item_ids[0]
    ratings_url = "https://shopee.sg/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0"

    offset = 0
    d = {"username": [], "rating": [], "comment": []}
    while True:
        data = requests.get(
            ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset)
        ).json()

        i = 1
        for i, rating in enumerate(data["data"]["ratings"], 1):
            d["username"].append(rating["author_username"])
            d["rating"].append(rating["rating_star"])
            d["comment"].append(rating["comment"])
            

        if i % 20:
            break

        offset += 20
        
    # Finished looping that itemid
    clear_output(wait=True)
    print(f"{round(idx/len(item_ids)*100,2)}% done")
    
df = pd.DataFrame(d)
print(f"{len(df)} reviews collated")
df.to_csv("data.csv", index=False)

99.77% done
886 reviews collated


In [8]:
df_reviews = pd.read_csv("data.csv")

In [9]:
df_reviews

,username,rating,comment
0,s*****y,5,Gorgeous tanzanite pieces! Quality of the jewe...
1,s*****8,5,Well received via my doorstep. Have been buyin...
2,n*****9,5,Received the bracelet within in 6 days. Love i...
3,s*****y,5,Gorgeous tanzanite pieces! Quality of the jewe...
4,merielyng.salvador,5,"Super nice and elegant, worth it and very good..."
...,...,...,...
575,woomien,5,NaN
576,sgoh20,5,NaN
577,stalky79,5,NaN
578,pxdpolesg,5,NaN


## Get itemids from Jamstones

In [3]:
import requests

shopid = "57629270"
d = {"item_id":[], 'name':[]}
newest = 0

while len(d['name']) == 0 or len(d['name']) % 100 == 0:

    url = f"https://shopee.sg/api/v4/search/search_items?by=pop&limit=100&match_id=57629270&newest={newest}&order=desc&page_type=shop&scenario=PAGE_OTHERS&version=2"

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://shopee.tw/shop/1819984/search?shopCollection=9271157',
    }    

    r = requests.get(url, headers=headers)
    data = r.json()
    newest += len(data['items'])
    
    for item in data['items']:
        d['item_id'].append(item['itemid'])
        d['name'].append(item['item_basic']['name'])

print(len(d['name']))

429


In [4]:
pd.DataFrame(d).to_csv("item_ids.csv")

In [6]:
df_items = pd.read_csv("item_ids.csv")

In [7]:
df_items

,Unnamed: 0,item_id,name
0,0,11355326722,*PAYMENT * Live Sale $1 Listing [Jamstones Lav...
1,1,11335219964,[Jamstones x Lavval] Healing Crystal Tree
2,2,11855320485,[Jamstones x Lavval] Wealth Money Bag Crystal ...
3,3,12921114204,[Jamstones x Lavval] Miffy Rabbit Crystal Resin
4,4,8185645925,[Jamstones x Lavval] Sugar & Spice Necklace (L...
...,...,...,...
424,424,14959078466,[Jamstones X Lavval] Leia Pixiu
425,425,14661872336,[Jamstones x Lavval] True Love
426,426,11623040204,[Jamstones x Lavval] Clear Quartz Solitaire (H...
427,427,8698649395,[Jamstones X Lavval] Elena Pixiu
